### Deep Learning in the Eye Tracking World 
#### the tutorial presented during ETRA 2021 (https://etra.acm.org/2021/acceptedtutorials.html)
#### the code downloaded from: https://github.com/kasprowski/etra2021
@author: pawel@kasprowski.pl


# Model transforming words into a number
- input: sequence of letters
- output: number 


Examples: 
- input: sequence 'sto' output: 100
- input: sequence 'dwieście czternaście' output: 214


In [3]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
import numpy as np
from number2word import getWords

## Model

In [4]:
model = Sequential()
model.add(LSTM(128,input_shape=(None,1),return_sequences=True)) # sequences of singlen numbers
model.add(LSTM(128))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer="adam",metrics=['mae','mse'])
num_epochs = 0
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, None, 128)         66560     
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 198,273
Trainable params: 198,273
Non-trainable params: 0
_________________________________________________________________


## Dataset creation

### Helper methods

In [5]:
# helper method, converts sequence of numbers to text
def to_text(sample):
    return ''.join([idx2char[int(x)] for x in sample])
# helper method, converts text to sequence of numbers
def to_number(words):
    return np.array([char2idx[char] for char in words])

### Dataset - **samples** and **labels**

In [6]:
DATASET_SIZE=200

samples = []
labels = []
all_words = ''
max_len = 0
for i in range(DATASET_SIZE):
    labels.append(i)
    #words = lslownie(i)
    words = getWords(i)
    samples.append(words)
    all_words += words
    if len(words)>max_len: 
        max_len = len(words)
  
print('Max len of text',max_len)
vocab = sorted(set(all_words))
vocab_size = len(vocab)
print('vocabulary (used letters)',vocab)
print ('unique characters',vocab_size)

Max len of text 27
vocabulary (used letters) [' ', 'd', 'e', 'f', 'g', 'h', 'i', 'l', 'n', 'o', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
unique characters 19


#### Creating a mapping from unique characters to indices

In [7]:
char2idx = {char:index for index, char in enumerate(vocab)}
print('char2idx:\n',char2idx)
idx2char = np.array(vocab)
print('idx2char\n',idx2char)

char2idx:
 {' ': 0, 'd': 1, 'e': 2, 'f': 3, 'g': 4, 'h': 5, 'i': 6, 'l': 7, 'n': 8, 'o': 9, 'r': 10, 's': 11, 't': 12, 'u': 13, 'v': 14, 'w': 15, 'x': 16, 'y': 17, 'z': 18}
idx2char
 [' ' 'd' 'e' 'f' 'g' 'h' 'i' 'l' 'n' 'o' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y'
 'z']


#### Convert letters to numbers using char2idx

In [8]:
samples_int = []
for s in samples:
    v = np.array([char2idx[char] for char in s])
    samples_int.append(v) # different sizes!
print(samples[123],' ->becomes-> ',samples_int[123])

one hundred twenty three    ->becomes->  [ 9  8  2  0  5 13  8  1 10  2  1  0 12 15  2  8 12 17  0 12  5 10  2  2
  0  0]


#### From list of lists to numpy - must have a fixed number of characters (30 -> max_len)

In [9]:
samples = np.zeros((DATASET_SIZE,max_len))
for i in range(len(samples_int)):
    for j in range(len(samples_int[i])):
        samples[i,j] = np.array(samples_int[i][j]) # all not used have '0' which is ' '
print('SAMPLES\n\n',samples)
print(samples.shape)

SAMPLES

 [[18.  2. 10. ...  0.  0.  0.]
 [ 9.  8.  2. ...  0.  0.  0.]
 [12. 15.  9. ...  0.  0.  0.]
 ...
 [ 9.  8.  2. ...  0.  0.  0.]
 [ 9.  8.  2. ...  0.  0.  0.]
 [ 9.  8.  2. ...  0.  0.  0.]]
(200, 27)


In [10]:
samples = np.expand_dims(samples,axis=2) #add the third dimension
labels = np.array(labels,dtype=float)

print("Sample (for 123):\n",samples[123])
print("Sample decoded:",to_text(samples[123]))
print("Label (output):",labels[123])

print('samples shape',samples.shape)
print('labels shape',labels.shape)

Sample (for 123):
 [[ 9.]
 [ 8.]
 [ 2.]
 [ 0.]
 [ 5.]
 [13.]
 [ 8.]
 [ 1.]
 [10.]
 [ 2.]
 [ 1.]
 [ 0.]
 [12.]
 [15.]
 [ 2.]
 [ 8.]
 [12.]
 [17.]
 [ 0.]
 [12.]
 [ 5.]
 [10.]
 [ 2.]
 [ 2.]
 [ 0.]
 [ 0.]
 [ 0.]]
Sample decoded one hundred twenty three   
Label (output): 123.0
samples shape (200, 27, 1)
labels shape (200,)


In [11]:
TRAINING_SIZE = .5
from sklearn.model_selection import train_test_split
(trainSamples, testSamples, trainLabels, testLabels) = train_test_split(samples, labels,train_size=TRAINING_SIZE, random_state=1)
print('Training samples:',len(trainSamples),' test samples',len(testSamples))

Training samples: 100  test samples 100


In [13]:
EPOCHS=100
BATCH_SIZE = int(len(trainSamples)/4)
print('Training with',len(trainSamples),'samples',EPOCHS,'epochs and batch_size=',BATCH_SIZE)
for x in range(10):
    H = model.fit(trainSamples, trainLabels, epochs=EPOCHS,verbose=0,batch_size=BATCH_SIZE)
    num_epochs += EPOCHS
    print("\n{}/10 Epochs: {} - loss={:6.3f}, loss improvement={:6.3f}".
          format(x, num_epochs,H.history['loss'][-1], H.history['loss'][0]-H.history['loss'][-1]))
    check_model()
print("Done")

Training with 100 samples 100 epochs and batch_size= 25

0/10 Epochs: 200 - loss=3299.996, loss improvement=1343.468
text => [predicted value] error=[error]
fifty five                  =>  81.44 error = 26.44
one hundred forty six       =>  81.44 error = 64.56
one hundred sixteen         =>  81.44 error = 34.56
fifty two                   =>  81.44 error = 29.44
one hundred seven           =>  81.44 error = 25.56
fifty nine                  =>  81.44 error = 22.44
one hundred                 =>  81.44 error = 18.56
one hundred eighty seven    =>  81.44 error = 105.56
one hundred twenty five     =>  81.44 error = 43.56
seven                       =>  81.44 error = 74.44
Mean error = 51.63133

1/10 Epochs: 300 - loss=3111.068, loss improvement=184.271
text => [predicted value] error=[error]
sixty eight                 =>  92.05 error = 24.05
forty two                   =>  92.05 error = 50.05
one hundred fifty three     =>  92.05 error = 60.95
one hundred ninety eight    =>  92.05 error 

In [14]:
import random

def check_model(verbose=0,how_many=10):
    pred = model.predict(samples)
    print('text => [predicted value] error=[error]')
    error = []
    for i in range(len(pred)):
        res = samples[i]
        error.append(abs(i-pred[i]))
        if verbose==1:
            train = ''
            if i in trainLabels: train='[T]'
            print(i,to_text(res),'=> {:.2f} error = {:.2f}'.format(pred[i,0],abs(i-pred[i,0])),train)
    if verbose<1: # if not verbose just display 'how_many' random samples
        for i in range(how_many):        
            x = random.randrange(DATASET_SIZE)
            res = samples[x]
            print(to_text(res),'=>  {:.2f} error = {:.2f}'.format(pred[x,0],abs(x-pred[x,0])))      
    print('Mean error =',np.mean(error))        
    return np.mean(error)
check_model(1)

text => [predicted value] error=[error]
0 zero                        => 0.26 error = 0.26 [T]
1 one                         => 2.52 error = 1.52 [T]
2 two                         => 1.65 error = 0.35 [T]
3 three                       => 4.04 error = 1.04 [T]
4 four                        => 35.98 error = 31.98 
5 five                        => 9.30 error = 4.30 
6 six                         => 7.07 error = 1.07 [T]
7 seven                       => 7.00 error = 0.00 [T]
8 eight                       => 8.73 error = 0.73 [T]
9 nine                        => 7.32 error = 1.68 [T]
10 ten                         => 9.31 error = 0.69 [T]
11 eleven                      => 10.02 error = 0.98 
12 twelve                      => 1.37 error = 10.63 
13 thirteen                    => 10.08 error = 2.92 
14 fourteen                    => 49.76 error = 35.76 
15 fifteen                     => 15.17 error = 0.17 [T]
16 sixteen                     => 44.82 error = 28.82 
17 seventeen                 

6.121855

In [13]:
x = to_number('two hundred seventy two') 
#              xxxxxxxxxxxxxxxxxxxxxxxxxxxxx
x = np.expand_dims(x,axis=1)
x = np.expand_dims(x,axis=0)
model.predict(x)

array([[-0.23773912]], dtype=float32)

In [ ]:
model.save('model_words2numbers.h5')